In [1]:
import pandas as pd

train = pd.read_csv('../preprocessed/train_df.csv')
test = pd.read_csv('../preprocessed/test_df.csv')


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5349 entries, 0 to 5348
Data columns (total 39 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   list_price                             5349 non-null   float64
 1   price_reduced_amount                   5349 non-null   float64
 2   description.year_built                 5349 non-null   int64  
 3   description.baths_3qtr                 5349 non-null   float64
 4   description.sold_price                 5349 non-null   float64
 5   description.baths_full                 5349 non-null   float64
 6   description.baths_half                 5349 non-null   float64
 7   description.lot_sqft                   5349 non-null   float64
 8   description.sqft                       5349 non-null   float64
 9   description.baths                      5349 non-null   float64
 10  description.sub_type                   5349 non-null   object 
 11  desc

In [3]:
from sklearn.model_selection import train_test_split

X = train[['description.sqft','city_mean_price','state_mean_price','description.year_built']]
y = train['description.sold_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
print(y_train.mean())


390365.61790137884


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=3)),
    ('lasso', Lasso())
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(pipeline.score(X_test, y_test))
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

0.6681659011925758
37395183891.70136
119214.18488089835
0.6681659011925758


/opt/anaconda3/envs/LHL/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.036e+14, tolerance: 5.798e+10
  model = cd_fast.enet_coordinate_descent(


In [6]:
from sklearn.svm import SVR
# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=3)),
    ('lasso', SVR())
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(pipeline.score(X_test, y_test))
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

-0.0365417452550556
116810386016.81717
204566.72430116346
-0.0365417452550556


In [7]:
from sklearn.ensemble import RandomForestRegressor
# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=2)),
    ('rf', RandomForestRegressor())
])
# Fit the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(pipeline.score(X_test, y_test))
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

0.926096561480704
8328356499.800102
24795.017330632792
0.926096561480704


In [15]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=3)),
    ('xgb', XGBRegressor())
])

# Define the parameter grid
params = {
    'xgb__n_estimators': [100, 200, 300],
    'xgb__max_depth': [3, 4, 5]
}

# Set up GridSearchCV with the correct scoring parameter
grid_search = GridSearchCV(pipeline, params, cv=5, scoring='neg_mean_absolute_error')

# Fit the pipeline
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the model using the best model
y_pred = best_model.predict(X_test)
print('Best Parameters:', best_params)
print(best_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

Best Parameters: {'xgb__max_depth': 5, 'xgb__n_estimators': 300}
0.9711814489953852
MSE: 3247631928.6745763
MAE: 14831.569141537675
R2: 0.9711814489953852
